In [11]:
# Initialize and check health
from client import R2RBackend

client = R2RBackend()
response = client.health()
response

AttributeError: 'R2RBackend' object has no attribute 'get_prompts'

In [13]:
from r2r import R2RClient
client = R2RClient('http://localhost:7272')
client.get_all_prompts()['results']

NameError: name 'R2RClient' is not defined

In [8]:
# File ingestion
from pathlib import Path
from utility.file_utils import iterate_over_files

folder_path = Path('data')
files = list(iterate_over_files(folder_path))
client.ingest_files(files) # Does not return anything. Logs can be traced to see which files were not ingested.

2024-10-25 19:55:48,860 - WARNING - client - [data\2005.11401v4.pdf] has already been ingested!
2024-10-25 19:55:49,141 - WARNING - client - [data\2009.02252v4.pdf] has already been ingested!
2024-10-25 19:55:49,391 - WARNING - client - [data\2309.15217v1.pdf] has already been ingested!
2024-10-25 19:55:49,663 - WARNING - client - [data\2311.09476v2.pdf] has already been ingested!
2024-10-25 19:55:50,010 - WARNING - client - [data\2312.10997v5.pdf] has already been ingested!
2024-10-25 19:55:50,298 - WARNING - client - [data\2401.05856v1.pdf] has already been ingested!
2024-10-25 19:55:50,580 - WARNING - client - [data\2405.07437v2.pdf] has already been ingested!
2024-10-25 19:55:50,915 - WARNING - client - [data\2406.14783v2.pdf] has already been ingested!
2024-10-25 19:55:51,201 - WARNING - client - [data\2407.01219v1.pdf] has already been ingested!
2024-10-25 19:55:51,467 - WARNING - client - [data\2408.02545v1.pdf] has already been ingested!
2024-10-25 19:55:51,789 - WARNING - clie

In [4]:
# Chunks ingestion
urls = [
    "https://docs.streamlit.io/deploy/tutorials/docker#create-a-dockerfile", 
    "https://medium.com/@ypredofficial/rag-based-conversational-chatbot-using-streamlit-364c4c02c2f1", 
    "https://blog.gopenai.com/building-a-multi-pdf-rag-chatbot-langchain-streamlit-with-code-d21d0a1cf9e5",
    "https://medium.com/@pritubera/building-a-streamlit-chatbot-for-technical-questions-with-llamaindex-and-openai-26808c841dab",
    "https://r2r-docs.sciphi.ai/cookbooks/ingestion",
    "https://docs.streamlit.io/get-started/fundamentals/advanced-concepts",
    "https://de.wikipedia.org/wiki/Deutschland"
]

from utility.scraper import Scraper
from utility.splitter import Splitter

scraper = Scraper()
splitter = Splitter()

documents = scraper.fetch_documents(urls)
split_documents = splitter.split_documents(documents)

if split_documents is None:
    print("No chunks found for any URL!")
    exit()

# Go over all the urls.
for url in urls:
    chunks = [split_doc for split_doc in split_documents if split_doc.metadata['source'] == url]
    # If chunks are found for the corresponding url ingest them.
    if chunks:
        metadata = chunks[0].metadata
        chunks_text = [{"text": chunk.page_content} for chunk in chunks]
        try:
            resp = client.ingest_chunks(chunks_text, metadata)
            print(resp)
        except Exception as e:
            pass
    else:
        print(f"No chunks found for [{url}]!")
        continue 

2024-10-25 13:35:57,982 - ERROR - client - Failed to ingest chunks for: [https://docs.streamlit.io/deploy/tutorials/docker#create-a-dockerfile]!
2024-10-25 13:35:58,226 - ERROR - client - Failed to ingest chunks for: [https://medium.com/@ypredofficial/rag-based-conversational-chatbot-using-streamlit-364c4c02c2f1]!
2024-10-25 13:35:58,488 - ERROR - client - Failed to ingest chunks for: [https://blog.gopenai.com/building-a-multi-pdf-rag-chatbot-langchain-streamlit-with-code-d21d0a1cf9e5]!
2024-10-25 13:35:58,764 - ERROR - client - Failed to ingest chunks for: [https://medium.com/@pritubera/building-a-streamlit-chatbot-for-technical-questions-with-llamaindex-and-openai-26808c841dab]!
2024-10-25 13:35:59,016 - ERROR - client - Failed to ingest chunks for: [https://r2r-docs.sciphi.ai/cookbooks/ingestion]!
2024-10-25 13:35:59,281 - ERROR - client - Failed to ingest chunks for: [https://docs.streamlit.io/get-started/fundamentals/advanced-concepts]!
2024-10-25 13:35:59,537 - ERROR - client - F

In [22]:
# Get metadata of all embedded files/documents
documents_data = client.documents_overview()

for data in documents_data:
    if data['title'] == 'dummy.txt':
        doc_id = data['id']
        break
    
# Do this after making changes to the file, ie. write something new.
update_resp = client.update_files(['data\\dummy.txt'], [doc_id])
print(f"Updated files count: {update_resp}")

# Make sure it has new content and print it out.
updated_file_chunks = client.document_chunks(doc_id)
[print(chunk['text']) for chunk in updated_file_chunks]

Updated files count: 1
My updated dummy.


[None]

In [7]:
# Get metadata of all embedded files/documents
documents_data = client.documents_overview()
documents_data

[{'id': '3dbfc547-0b09-51a5-add1-4a100f18d995',
  'title': 'computer-networks-global-edition-6nbsped-1292374063-9781292374062_compress.pdf',
  'user_id': '2acb499e-8428-543b-bd85-0d9098718220',
  'type': 'pdf',
  'created_at': '2024-10-25T17:32:59.009722Z',
  'updated_at': '2024-10-25T17:32:59.009729Z',
  'ingestion_status': 'parsing',
  'kg_extraction_status': 'pending',
  'version': 'v0',
  'collection_ids': [],
  'metadata': {'version': 'v0'}},
 {'id': '4fa60814-15cc-5bc2-8b2c-961edba43bff',
  'title': 'dummy.txt',
  'user_id': '2acb499e-8428-543b-bd85-0d9098718220',
  'type': 'txt',
  'created_at': '2024-10-25T11:42:43.747409Z',
  'updated_at': '2024-10-25T11:48:38.820632Z',
  'ingestion_status': 'success',
  'kg_extraction_status': 'pending',
  'version': 'v5',
  'collection_ids': ['122fdf6a-e116-546b-a8f6-e4cb2e2c0a09'],
  'metadata': {'title': 'dummy.txt', 'version': 'v5'}},
 {'id': 'c8796189-b719-564d-be6d-9487b58d8dc4',
  'title': 'Deutschland â€“ Wikipedia',
  'user_id': '2ac

In [5]:
client.delete({"document_id": {"$eq": '3dbfc547-0b09-51a5-add1-4a100f18d995'}})

2024-10-25 19:54:44,987 - WARNING - client - Could not delete a file with following filter: document_id!


0

In [11]:
# Get a particular documents chunks based on id
doc_id = documents_data[0]['id']

# Exits
doc_chunks = client.document_chunks(doc_id)

# Doesn't
from uuid import uuid4
try:
    client.document_chunks(uudi4())
except Exception as e:
    pass

for chunk in doc_chunks:
    print(chunk['text'])

, den Vorrang der Gemeindeebene vor der Kreisebene nach MaÃŸgabe der Gesetze festgestellt: Hiernach gilt fÃ¼r StÃ¤dte und Gemeinden â€�als Essentialeâ€œ und â€�identitÃ¤tsbestimmendes Merkmal der gemeindlichen Selbstverwaltungâ€œ das Prinzip der â€�â€šUniversalitÃ¤tâ€˜ des gemeindlichen Wirkungskreisesâ€œ,[55] im Gegensatz zur speziellen Kompetenz der GemeindeverbÃ¤nde kraft ausdrÃ¼cklicher gesetzlicher Zuweisung
, womit es auch keine feststehenden Gemeindeverbandshoheiten gibt.[56] Siehe auch: Kommunale Selbstverwaltung (Deutschland) Land Hauptstadt FlÃ¤chein kmÂ²[2] Einwohner Einwohnerpro kmÂ² Baden-WÃ¼rttemberg Baden-WÃ¼rttemberg Stuttgart 035.748 11.339.260[57] 0.317 Bayern Bayern MÃ¼nchen 070.541 13.435.062[58] 0.190 Berlin Berlin â€” 000.891 03.782.202[59] 4.241 Brandenburg Brandenburg Potsdam 029.654 02.581.667[60] 0.087 Bremen Bremen Bremen 000.420 00.691.703[61] 1648 Hamburg Hamburg â€” 000.755
. Frankreich annektierte den deutschen Westen und Nordwesten und schuf deutsche Vas

In [7]:
query = "Can you be very explicit and provide a comprehensive answer as to what RAGAs is and how it works?"
LLM_resp = client.rag(query=query)

2024-10-25 14:03:10,839 - INFO - client - Can you be very explicit and provide a comprehensive answer as to what RAGAs is and how it works?


In [10]:
print(LLM_resp['completion']['choices'][0]['message']['content'].replace('\n', ' '))

Based on the provided context, I will attempt to provide an explicit and comprehensive answer to the query.  RAGAS (Retrieval Augmented Generation Assessment System) is a framework for the automated assessment of Retrieval-Augmented-Generation (RAG) systems. According to [3], RAGAS provides an integration with both llama-index and Langchain, the most widely used frameworks for building RAG solutions, thus enabling developers to easily integrate RAGAS into their standard workflow.  To understand how RAGAS works, let's break down the concept of RAG first. A RAG system is an information retrieval approach designed to overcome the limitations of using a Large Language Model (LLM) directly [4]. RAG works by taking a natural language query and converting it into an embedding which is used to semantically search a set of documents. Retrieved documents are then passed to a large language model to generate an answer.  The process can be divided into three main components: Indexing, Retrieval, a